In [1]:
import numpy as np
import os
import pandas as pd
import assembly as assembly
import numpy as np
import matplotlib.pyplot as plt
import glob
import pickle

#### Specify the path where you put the generated data from previous script


In [25]:
path = "/media/adrian/6aa1794c-0320-4096-a7df-00ab0ba946dc/Cell_assembly_yuqin/data_10-11_10_2019_OD_Rn3_SD1"
os.chdir(path)

In [26]:
def load_data():
    load_dict = {}
    for np_name in glob.glob('*.np[yz]'):
        load_dict[np_name[:-4]] = np.load(np_name)
    return load_dict

def clean_actmat(load_dict):
    del_indices = [i for i, x in enumerate(load_dict["Trial1_neuron_name"]) if x == "null"]
    mat_dict = dict([(key, value) for key, value in load_dict.items() if '_actmat' in key.lower()])
    actmat_clean = {}
    for each_key, each_mat in mat_dict.items():
        new_mat = np.delete(each_mat,del_indices, 0)
        actmat_clean[each_key[:len(each_key) - 7]] = new_mat
    neuron_name =  dict(enumerate([x for x in load_dict["Trial1_neuron_name"] if x != "null"]))
#     neuron_name =  [x for x in load_dict["Trial1_neuron_name"] if x != "null"]
    return actmat_clean, neuron_name

In [27]:
load_dict = load_data();
actmat_dict, neuron_name_dict = clean_actmat(load_dict)

In [28]:
neuron_name_dict

{0: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T6_UID0',
 1: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T8_UID2',
 2: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T8_UID5',
 3: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID0',
 4: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID2',
 5: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID4',
 6: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID53',
 7: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID61',
 8: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID65',
 9: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID2',
 10: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID3',
 11: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID6',
 12: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID7',
 13: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID15',
 14: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID33',
 15: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID37',
 16: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID1',
 17: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID3',
 18: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID4',
 19: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID5',
 20: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID8',
 21: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID20',


In [30]:
actmat_dict

{'Post_Trial1': array([[1., 1., 0., ..., 2., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'Trial2': array([[1., 2., 3., ..., 3., 1., 3.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'Trial1': array([[1., 2., 1., ..., 0., 2., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]]),
 'Trial4': array([[2., 1., 1., ..., 1., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],


In [73]:
def get_interneurons_list(txt_file_name, project_name, condition_name, study_day_name, rat_name):
    inter_neurons = open(txt_file_name, "r")
    inter_neurons = list(inter_neurons.read().split(','))
    if '\n' in inter_neurons: inter_neurons.remove('\n')
    if '' in inter_neurons: inter_neurons.remove('')
    interneurons_list = []
    for interneuron in inter_neurons:
        study_day_interneuron = interneuron[interneuron.find("SD") : interneuron.find("_T")]
#         print(study_day_interneuron)
        study_day_match = study_day_interneuron == study_day_name
#         print(study_day_match)
        if (project_name in interneuron) and (condition_name in interneuron) and study_day_match and (rat_name in interneuron):
            interneurons_list.append(interneuron[: len(interneuron) - 3])
    return interneurons_list, inter_neurons

In [74]:
inter_neurons_RGS14, interneuron_old_RGS14 = get_interneurons_list("inter_neurons_RGS14.txt", 
                                                                   'RGS_PROJECT', 'OD', 'SD1', 'Rn3')
inter_neurons_vehicle, interneuron_old_vehicle = get_interneurons_list("inter_neurons_vehicle.txt",
                                                                        'RGS_PROJECT', 'OD', 'SD1', 'Rn3')

In [75]:
inter_neurons_RGS14, inter_neurons_vehicle

(['RGS_PROJECT_RGS14_PFC_Rn3_OD_SD1_T10_UID7',
  'RGS_PROJECT_RGS14_PFC_Rn3_OD_SD1_T12_UID4',
  'RGS_PROJECT_RGS14_PFC_Rn3_OD_SD1_T13_UID3',
  'RGS_PROJECT_RGS14_PFC_Rn3_OD_SD1_T9_UID4'],
 [])

In [77]:
neuron_name_dict

{0: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T6_UID0',
 1: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T8_UID2',
 2: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T8_UID5',
 3: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID0',
 4: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID2',
 5: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID4',
 6: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID53',
 7: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID61',
 8: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID65',
 9: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID2',
 10: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID3',
 11: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID6',
 12: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID7',
 13: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID15',
 14: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID33',
 15: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID37',
 16: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID1',
 17: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID3',
 18: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID4',
 19: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID5',
 20: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID8',
 21: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID20',


In [80]:
def find_index_interneurons(interneurons, name_dict):
    index_list = []
    for interneuron in interneurons:
        for  index, neuron_name in name_dict.items():
            y = neuron_name[neuron_name.find("_T") : ]
            x = interneuron[interneuron.find("_T") :]
#             print(x)
#             print(y)
            match = x == y
            if match:
                index_list.append(index)
    return index_list

In [81]:
interneuron_index_list = find_index_interneurons(inter_neurons_RGS14, neuron_name_dict)
interneuron_index_list

[12, 18, 25, 5]

In [82]:
def remove_interneuron(interneuron_index_list, neuron_name_dict, actmat_dict):
    load_dict = {}
    for key, actmat in actmat_dict.items():
        actmat_new = np.delete(actmat, interneuron_index_list, 0)
        load_dict[key] = actmat_new
    
    neuron_name_list = []
    for i in range(len(neuron_name_dict)):
        if i not in interneuron_index_list:
            neuron_name_list.append(neuron_name_dict[i])
    neuron_name_new =  dict(enumerate([x for x in neuron_name_list]))
#     print(neuron_name_new)
    return load_dict, neuron_name_new

In [83]:
actmat_dict_new, neuron_name_dict_new = remove_interneuron(interneuron_index_list, neuron_name_dict, actmat_dict)

In [84]:
actmat_dict_new

{'Post_Trial1': array([[1., 1., 0., ..., 2., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'Trial2': array([[1., 2., 3., ..., 3., 1., 3.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'Trial1': array([[1., 2., 1., ..., 0., 2., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]]),
 'Trial4': array([[2., 1., 1., ..., 1., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],


In [85]:
actmat_dict_new['Post_Trial1'].max()

6.0

In [86]:
neuron_name_dict_new

{0: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T6_UID0',
 1: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T8_UID2',
 2: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T8_UID5',
 3: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID0',
 4: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID2',
 5: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID53',
 6: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID61',
 7: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID65',
 8: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID2',
 9: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID3',
 10: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID6',
 11: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID15',
 12: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID33',
 13: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID37',
 14: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID1',
 15: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID3',
 16: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID5',
 17: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID8',
 18: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID20',
 19: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID24',
 20: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T13_UID0',
 21: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T13_UID2'

In [87]:
neuron_name_dict_new

{0: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T6_UID0',
 1: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T8_UID2',
 2: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T8_UID5',
 3: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID0',
 4: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID2',
 5: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID53',
 6: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID61',
 7: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T9_UID65',
 8: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID2',
 9: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID3',
 10: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID6',
 11: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID15',
 12: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID33',
 13: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T10_UID37',
 14: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID1',
 15: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID3',
 16: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID5',
 17: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID8',
 18: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID20',
 19: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T12_UID24',
 20: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T13_UID0',
 21: 'RGS_PROJECT_RGS14_OD_Rn3_SD1_T13_UID2'

In [88]:
'''The file will saved in the path previously defined'''

act_mat_file = open("actmat_dict.pkl", "wb")
pickle.dump(actmat_dict_new, act_mat_file)                     
act_mat_file.close()

neuron_name_file = open("neuron_name_dict.pkl", "wb")
pickle.dump(neuron_name_dict_new, neuron_name_file)
neuron_name_file.close()
